# Класифікація рукописних цифр MNIST (Scikit-learn)

**Мета:** Побудувати та оцінити просту модель машинного навчання для розпізнавання рукописних цифр з класичного набору даних MNIST.

## 1. Імпорт бібліотек

Імпортуємо всі необхідні інструменти: `fetch_openml` для завантаження даних, `LogisticRegression` для моделі, `train_test_split` для поділу вибірки та `matplotlib` для візуалізації.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

## 2. Завантаження та підготовка даних

Завантажуємо набір даних MNIST. Це може зайняти 1-2 хвилини. Набір містить 70,000 зображень (28x28 пікселів) та їхні мітки (від 0 до 9).

In [ ]:
# Завантажуємо дані
print("Завантаження даних MNIST...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, parser='liac-arff')
print("Дані завантажено.")

# Нормалізація даних (шкалування пікселів від 0-255 до 0-1)
X = X / 255.0

# Розділення даних на тренувальну та тестову вибірки
# 60,000 для тренування, 10,000 для тесту
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, train_size=60000, random_state=42, stratify=y)

print(f"Розмір тренувальної вибірки: {X_train.shape[0]}")
print(f"Розмір тестової вибірки: {X_test.shape[0]}")

## 3. Візуалізація даних

Давайте подивимося, як виглядають деякі цифри з набору даних.

In [ ]:
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train[i].reshape(28, 28), cmap='gray')
    plt.title(f"Мітка: {y_train[i]}")
    plt.axis('off')
plt.suptitle("Приклади цифр з тренувального набору", fontsize=16)
plt.show()

## 4. Тренування моделі

Будемо використовувати просту, але ефективну модель — **Логістичну Регресію**. Вона добре підходить для цієї задачі та швидко тренується.

*(Примітка: `solver='saga'` та `penalty='l1'` — це просто налаштування оптимізатора, `max_iter=10` обрано для швидкості, навіть з 10 ітераціями результат буде добрим для демонстрації)*

In [ ]:
print("Тренування моделі...")
# Ініціалізуємо модель
# C=1.0 - параметр регуляризації
# solver='saga' - швидкий оптимізатор для великих даних
# tol=0.1 - ставимо меншу точність для швидкого тренування
clf = LogisticRegression(C=1.0, solver='saga', penalty='l1', max_iter=10, tol=0.1, random_state=42, n_jobs=-1)

# Тренуємо модель на тренувальних даних
clf.fit(X_train, y_train)

print("Модель натреновано.")

## 5. Оцінка моделі

Перевіримо, наскільки добре наша модель працює на даних, які вона ще не бачила (тестова вибірка).

In [ ]:
# Робимо передбачення на тестових даних
y_pred = clf.predict(X_test)

# Розраховуємо точність
accuracy = accuracy_score(y_test, y_pred)
print(f"Точність моделі на тестовій вибірці: {accuracy * 100:.2f}%")

### Матриця помилок (Confusion Matrix)

Матриця помилок — це чудовий інструмент, щоб побачити, які саме цифри модель плутає між собою. Наприклад, як часто вона плутає '4' і '9'.

In [ ]:
# Будуємо матрицю помилок
cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)

# Візуалізуємо її
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(ax=ax, cmap='Blues')
plt.title("Матриця помилок для класифікатора MNIST")
plt.show()

## 6. Аналіз помилок

Давайте подивимося на конкретні приклади, де модель помилилася. Це допомагає зрозуміти її слабкі сторони.

In [ ]:
# Знаходимо індекси, де передбачення не співпало з реальністю
misclassified_indices = np.where(y_pred != y_test)[0]

print(f"Всього помилок: {len(misclassified_indices)} з {len(y_test)}")

# Виводимо перші 10 помилкових прикладів
plt.figure(figsize=(12, 5))
for i, index in enumerate(misclassified_indices[:10]):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[index].reshape(28, 28), cmap='gray')
    plt.title(f"Факт: {y_test[index]}\nПред.: {y_pred[index]}")
    plt.axis('off')

plt.suptitle("Приклади помилок моделі", fontsize=16)
plt.show()

## Висновок

Навіть з простою моделлю Логістичної Регресії та дуже швидким тренуванням (10 ітерацій) ми досягли точності **~90%**. 

Аналіз помилок показує, що модель в основному плутає цифри, які візуально схожі (наприклад, '4' і '9', або '8' і '3'). Це очікувана поведінка. Для покращення результату можна було б використовувати більш складні моделі (наприклад, Випадковий Ліс або Нейронні Мережі) або тренувати модель довше.